In [62]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


In [5]:
url = "https://db.netkeiba.com/?sort_key=date&sort_type=desc&pid=race_list&serial=a%3A15%3A%7Bs%3A3%3A%22pid%22%3Bs%3A9%3A%22race_list%22%3Bs%3A4%3A%22word%22%3Bs%3A0%3A%22%22%3Bs%3A5%3A%22track%22%3Ba%3A2%3A%7Bi%3A0%3Bs%3A1%3A%221%22%3Bi%3A1%3Bs%3A1%3A%222%22%3B%7Ds%3A10%3A%22start_year%22%3Bs%3A4%3A%22none%22%3Bs%3A9%3A%22start_mon%22%3Bs%3A4%3A%22none%22%3Bs%3A8%3A%22end_year%22%3Bs%3A4%3A%22none%22%3Bs%3A7%3A%22end_mon%22%3Bs%3A4%3A%22none%22%3Bs%3A9%3A%22kyori_min%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22kyori_max%22%3Bs%3A0%3A%22%22%3Bs%3A4%3A%22sort%22%3Bs%3A4%3A%22date%22%3Bs%3A4%3A%22list%22%3Bs%3A2%3A%2220%22%3Bs%3A9%3A%22style_dir%22%3Bs%3A17%3A%22style%2Fnetkeiba.ja%22%3Bs%3A13%3A%22template_file%22%3Bs%3A14%3A%22race_list.html%22%3Bs%3A9%3A%22style_url%22%3Bs%3A18%3A%22%2Fstyle%2Fnetkeiba.ja%22%3Bs%3A6%3A%22search%22%3Bs%3A42%3A%22%B6%A5%C1%F6%BC%EF%CA%CC%5B%BC%C7%A1%A2%A5%C0%A1%BC%A5%C8%5D%A1%A2%B4%FC%B4%D6%5B%CC%B5%BB%D8%C4%EA%A1%C1%CC%B5%BB%D8%C4%EA%5D%22%3B%7D&page=28693"

In [6]:
response = requests.get(url)

In [9]:
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
next_page_post_parameter = None
next_page_element = soup.select_one("div.pager a:contains('次')")


In [ ]:
if next_page_element:
    RACE_LISTING_PAGE_POST_INPUT_DIC["page"] = RACE_LISTING_PAGE_POST_INPUT_DIC.get(
        "page") + 1
    next_page_post_parameter = RACE_LISTING_PAGE_POST_INPUT_DIC


In [11]:
next_page_element

In [12]:
url = "https://db.netkeiba.com/race/202147010804/"

In [13]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [63]:
table = soup.find("table", summary="レース結果")
if table is None:
    print("None")
rows = table.find_all("tr")
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
df = pd.DataFrame(columns=columns)
# TODO: DataFrameにせず保持する
# 全行のうちのある行成分について
for i in range(len(rows)):
    # 全ての<td>タグ（セルデータ）を取得しtdsに格納、リスト化
    tds = rows[i].find_all('td')
    # tdsのデータ数がカラム数に一致しない場合（ブランク）などは排除し、
    if len(tds) == len(columns):
        # （ある行成分の）全セルデータをテキスト成分としてvaluesに格納、リスト化
        values = [td.text.replace('\n', '').replace(
            '\xa0', ' ') for td in tds]
        # valuesをpd.seriesデータに変換、データフレームに結合
        df = df.append(pd.Series(values, index=columns),
                       ignore_index=True)

In [64]:
df.head()

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ指数,...,上り,単勝,人気,馬体重,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円)
0,1,3,3,サニーシェルブール,セ4,56,吉井友彦,1:30.4,,**,...,39.7,3.1,2,444(-14),,,,[地]森山英雄,豊原純弥,27.0
1,2,2,2,チャーミングポポ,牝4,54,大原浩司,1:31.4,5,**,...,41.4,5.4,3,472(-1),,,,[地]森山英雄,村下義等,8.1
2,3,4,4,アービター,セ4,56,向山牧,1:31.7,1.1/2,**,...,39.9,6.5,4,455(+3),,,,[地]後藤正義,紙谷洋一,4.1
3,4,6,6,ナリノレッドスター,牡6,56,松本剛志,1:32.1,2,**,...,41.8,1.8,1,449(-3),,,,[地]加藤幸保,成塚清志,2.7
4,5,5,5,コスモグラシエ,牝4,54,森島貴之,1:32.4,1.1/2,**,...,40.2,37.3,5,433(-2),,,,[地]東川公則,杉浦和也,1.4


In [80]:
soup.find('table')

<table cellpadding="0" cellspacing="1" class="race_table_01 nk_tb_common" summary="レース結果">
<tr class="txt_c">
<th nowrap="nowrap">着<br/>順</th>
<th nowrap="nowrap">枠<br/>番</th>
<th nowrap="nowrap">馬<br/>番</th>
<th nowrap="nowrap">馬名</th>
<th nowrap="nowrap">性齢</th>
<th nowrap="nowrap">斤量</th>
<th nowrap="nowrap">騎手</th>
<th nowrap="nowrap">タイム</th>
<th nowrap="nowrap">着差</th>
<diary_snap_cut>
<th nowrap="nowrap">ﾀｲﾑ<br/>指数<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">通過</th>
<th nowrap="nowrap">上り</th>
</diary_snap_cut>
<th nowrap="nowrap">単勝</th>
<th nowrap="nowrap">人<br/>気</th>
<th nowrap="nowrap">馬体重</th>
<diary_snap_cut>
<th nowrap="nowrap">調教<br/>ﾀｲﾑ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium.gif" width="16"/></th>
<th nowrap="nowrap">厩舎<br/>ｺﾒﾝﾄ<br/><img height="16" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/ico_premium

In [91]:
horse_detail_dict = {}
for i in soup.find('table').find_all("th"):
    horse_detail_dict[i.text] = []

In [92]:
horse_detail_dict

{'着順': [],
 '枠番': [],
 '馬番': [],
 '馬名': [],
 '性齢': [],
 '斤量': [],
 '騎手': [],
 'タイム': [],
 '着差': [],
 'ﾀｲﾑ指数': [],
 '通過': [],
 '上り': [],
 '単勝': [],
 '人気': [],
 '馬体重': [],
 '調教ﾀｲﾑ': [],
 '厩舎ｺﾒﾝﾄ': [],
 '備考': [],
 '調教師': [],
 '馬主': [],
 '賞金(万円)': []}

In [100]:
list(horse_detail_dict.keys())[0]

'着順'

In [106]:
horse_detail_dict = {}
for i in soup.find('table').find_all("th"):
    horse_detail_dict[i.text] = []
for i in soup.find('table').find_all('tr'):
    count = 0
    tmp = i.find_all("td")
    for i in tmp:
        horse_detail_dict[list(horse_detail_dict.keys())[count]].append(i)
        count += 1
        

In [126]:
import sys

In [127]:
sys.getrecursionlimit()

3000

In [130]:
sys.setrecursionlimit(10000)

In [117]:
import pickle

In [123]:
path = "/Users/daikimiyazaki/workspace/pndnism/horse_race_prediction/horse_info_crawler/tests/race/normalizer/data/test_race_info_dict.pickle"

In [120]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

In [131]:
with open(path, mode='wb') as f:
    pickle.dump(horse_detail_dict,f)

In [132]:
with open(path, mode='rb') as f:
    data = pickle.load(f)

In [133]:
data

{'着順': [<td class="txt_r" nowrap="nowrap">1</td>,
  <td class="txt_r" nowrap="nowrap">2</td>,
  <td class="txt_r" nowrap="nowrap">3</td>,
  <td class="txt_r" nowrap="nowrap">4</td>,
  <td class="txt_r" nowrap="nowrap">5</td>,
  <td class="txt_r" nowrap="nowrap">6</td>,
  <td class="txt_r" nowrap="nowrap">7</td>,
  <td class="txt_r" nowrap="nowrap">8</td>],
 '枠番': [<td align="right" class="w3ml" nowrap="nowrap"><span>3</span></td>,
  <td align="right" class="w2ml" nowrap="nowrap"><span>2</span></td>,
  <td align="right" class="w4ml" nowrap="nowrap"><span>4</span></td>,
  <td align="right" class="w6ml" nowrap="nowrap"><span>6</span></td>,
  <td align="right" class="w5ml" nowrap="nowrap"><span>5</span></td>,
  <td align="right" class="w8ml" nowrap="nowrap"><span>8</span></td>,
  <td align="right" class="w7ml" nowrap="nowrap"><span>7</span></td>,
  <td align="right" class="w1ml" nowrap="nowrap"><span>1</span></td>],
 '馬番': [<td class="txt_r" nowrap="nowrap">3</td>,
  <td class="txt_r" nowr

In [112]:
import json

In [115]:
json_open = open("/Users/daikimiyazaki/workspace/pndnism/horse_race_prediction/horse_info_crawler/tests/race/normalizer/data/test_race_info_dict.json", 'r')

In [116]:
json_load = json.load(json_open)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [74]:
table.find_all("th")[]

<th nowrap="nowrap">斤量</th>

In [18]:
for td in table.find_all('td'):
    data.append(td.text.strip().rstrip(","))